In [2]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [207]:
import os
import numpy as np
import random

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [208]:
ENV_NAME = 'TRIA-RL-MODEL3D'

In [633]:
TINI= 50.0; HINI= 20.0; AINI= 1000 

TMIN =-40.0; TMAX=120; HMIN=0.0; HMAX=100.0; AMIN=0.0; AMAX=20000.0

EQUI_LEN= 30

R1= -1; R2= -2; R3= 3; NR3= -3

CONST_WAIGHT_VEC = [.25,.25,.25,.25]

D = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

In [659]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([2,2,2,2,2]))
        
        self.observation_space = Box(low=np.array([TMIN, HMIN, AMIN]), high=np.array([TMAX, HMAX, AMAX]))
        
        self.state = [TINI + random.uniform(-2.0,2.0), HINI + random.uniform(-2.0,2.0), AINI + random.uniform(-2.0,2.0)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_length = EQUI_LEN
        
    def step(self, action):
        
        ap = [1 if e == 1 else -1 for e in action]
        
        actionPrime = [a*b for a,b in zip(ap, CONST_WAIGHT_VEC)]
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_length -= 1
        
        reward = [R3 if e >= D[i][0] and e<= D[i][1] else R2 if e >= D[i][2] and e<= D[i][3] else R1 if e >= D[i][4] and e <= D[i][5] else NR3 for i,e in enumerate(self.state)]
        #print('reward: ', reward, sum(reward))
        reward = sum(reward)
        #print('$$$', reward)
            
        if self.equilibrium_length <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, {}
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[TINI + random.uniform(-2.0,2.0),HINI + random.uniform(-2.0,2.0),AINI + random.uniform(-2.0,2.0)]
        #print('@@@', self.state)
        self.equilibrium_length = EQUI_LEN
        
        return self.state, {}
        

In [660]:
env= TriaEnv()

In [661]:
env.observation_space.sample()

array([   21.042469,    69.578705, 10525.868   ], dtype=float32)

In [662]:
env.action_space.sample()

array([0, 0, 0, 0, 1])

In [663]:
env.state

[51.49377950496303, 18.016829206626475, 998.4755275125497]

In [664]:
#env.reset()

In [665]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

*** [-0.5, 0.0, 0.25] [49.27093914434053, 18.441779610406197, 1000.6161297430068]
reward:-3 state:[48.77093914434053, 18.441779610406197, 1000.8661297430068]
*** [0.5, 0.0, -0.25] [48.77093914434053, 18.441779610406197, 1000.8661297430068]
reward:-3 state:[49.27093914434053, 18.441779610406197, 1000.6161297430068]
*** [0.5, 0.0, 0.25] [49.27093914434053, 18.441779610406197, 1000.6161297430068]
reward:-3 state:[49.77093914434053, 18.441779610406197, 1000.8661297430068]
*** [0.0, 0.5, -0.25] [49.77093914434053, 18.441779610406197, 1000.8661297430068]
reward:-3 state:[49.77093914434053, 18.941779610406197, 1000.6161297430068]
*** [0.0, 0.0, -0.25] [49.77093914434053, 18.941779610406197, 1000.6161297430068]
reward:-3 state:[49.77093914434053, 18.941779610406197, 1000.3661297430068]
*** [-0.5, 0.0, -0.25] [49.77093914434053, 18.941779610406197, 1000.3661297430068]
reward:-3 state:[49.27093914434053, 18.941779610406197, 1000.1161297430068]
*** [0.5, 0.0, 0.25] [49.27093914434053, 18.94177961

In [648]:
log_path = os.path.join('training','logs')

In [649]:
#from stable_baselines3 import A2C
#from stable_baselines3.common.vec_env import SubprocVecEnv
#env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1) #, tensorboard_log=log_path)
#model = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=4000)

*** [0.0, 0.0, -0.25] [48.89338939069337, 20.390052816227882, 999.9674357324164]
reward:-4 state:[48.89338939069337, 20.390052816227882, 999.7174357324164]
*** [-0.5, -0.5, -0.25] [48.89338939069337, 20.390052816227882, 999.7174357324164]
reward:-3 state:[48.39338939069337, 19.890052816227882, 999.4674357324164]
*** [0.0, -0.5, -0.25] [48.39338939069337, 19.890052816227882, 999.4674357324164]
reward:-3 state:[48.39338939069337, 19.390052816227882, 999.2174357324164]
*** [-0.5, -0.5, -0.25] [48.39338939069337, 19.390052816227882, 999.2174357324164]
reward:-3 state:[47.89338939069337, 18.890052816227882, 998.9674357324164]
*** [0.0, -0.5, -0.25] [47.89338939069337, 18.890052816227882, 998.9674357324164]
reward:-3 state:[47.89338939069337, 18.390052816227882, 998.7174357324164]
*** [0.0, -0.5, -0.25] [47.89338939069337, 18.390052816227882, 998.7174357324164]
reward:-3 state:[47.89338939069337, 17.890052816227882, 998.4674357324164]
*** [-0.5, -0.5, -0.25] [47.89338939069337, 17.8900528162